## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:\\Users\\asus\\World_Weather_Analysis\\Weather_Database\\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chanute,US,37.68,-95.46,32.00,97,90,9.17,freezing rain
1,1,Hobart,AU,-42.88,147.33,48.20,93,75,4.70,broken clouds
2,2,Albany,US,42.60,-73.97,51.01,63,56,6.17,broken clouds
3,3,Pozo Colorado,PY,-23.49,-58.80,93.49,24,0,3.33,clear sky
4,4,Hermanus,ZA,-34.42,19.23,64.00,85,99,4.00,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp)&(city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)   

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Saint-Philippe,RE,-21.36,55.77,75.20,73,75,2.24,broken clouds
12,12,Bengkulu,ID,-3.80,102.27,75.96,86,95,3.51,light rain
13,13,Atuona,PF,-9.80,-139.03,77.99,77,45,22.93,light rain
16,16,Yaan,NG,7.38,8.57,83.12,62,9,0.76,clear sky
18,18,Lorengau,PG,-2.02,147.27,80.26,76,83,1.39,broken clouds
21,21,Kununurra,AU,-15.77,128.73,78.80,88,80,5.82,broken clouds
24,24,Rikitea,PF,-23.12,-134.97,75.11,85,73,16.53,broken clouds
32,32,Saint Marys,US,30.73,-81.55,82.00,81,73,7.00,broken clouds
36,36,Samarai,PG,-10.62,150.67,80.83,84,80,6.58,broken clouds
41,41,Zaraza,VE,9.35,-65.32,85.59,73,24,5.37,light rain


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                220
City                   220
Country                218
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [73]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = float("NaN")
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Saint-Philippe,RE,75.20,broken clouds,-21.36,55.77,NaN
12,Bengkulu,ID,75.96,light rain,-3.80,102.27,NaN
13,Atuona,PF,77.99,light rain,-9.80,-139.03,NaN
16,Yaan,NG,83.12,clear sky,7.38,8.57,NaN
18,Lorengau,PG,80.26,broken clouds,-2.02,147.27,NaN
21,Kununurra,AU,78.80,broken clouds,-15.77,128.73,NaN
24,Rikitea,PF,75.11,broken clouds,-23.12,-134.97,NaN
32,Saint Marys,US,82.00,broken clouds,30.73,-81.55,NaN
36,Samarai,PG,80.83,broken clouds,-10.62,150.67,NaN
41,Zaraza,VE,85.59,light rain,9.35,-65.32,NaN


In [74]:
hotel_df.count()

City                   218
Country                218
Max Temp               218
Current Description    218
Lat                    218
Lng                    218
Hotel Name               0
dtype: int64

In [81]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]  
    params["location"] = f"{lat},{lng}" 
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
      print(hotels["results"][0]["name"])
    except (IndexError):
      print("Hotel not found... skipping.")
    
hotel_df.head(20)      

Chambres d'hôte "La Trinité"
Grage Hotel Bengkulu
Villa Enata
Hotel not found... skipping.
Lorengau Harbourside Hotel
Hotel Kununurra
Pension Maro'i
Spencer House Inn
Nuli Sapi
Hotel Zaraza
Dandan’s Garden & Farm Resort
Hotel not found... skipping.
Swee Ann Homestay
Stop.
Emeraude Beach Attitude Hotel
Grand Star Guest House
Paradise Hotel
Pirineos
Hotel Tesoro Los Cabos
Osprey Beach Hotel
Conrad Bora Bora Nui
Erlyn’s Sta Rosa City Home
Orchid Main Camp Goubet
Hotel not found... skipping.
CHACARA BAILLY
Hotel Syariah CN Darussalam
Hotel not found... skipping.
Puri Brata
Sylvia Hotel Premier Kupang
Hôtel Le Dronmi
Hotels in Lakshadweep Islands
Idlers' Rest Beach Hotel
Cabañas Estancia Bayamon
Mini Hotel Central
Shandrani Beachcomber Resort & Spa
Hotel not found... skipping.
Pousada Monte Carmelo O Pão
Fortune Resort Bay Island
Cititel Penang
Hôtel Tshia-Muntu de Tshikapa
Camp Pasica
Atlantis Hotel
Bonthe Holiday Village
Isles Sunset Lodge
Ono Hot Dog Cart and Catering
Laura Visah Transie

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Saint-Philippe,RE,75.20,broken clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
12,Bengkulu,ID,75.96,light rain,-3.80,102.27,Grage Hotel Bengkulu
13,Atuona,PF,77.99,light rain,-9.80,-139.03,Villa Enata
16,Yaan,NG,83.12,clear sky,7.38,8.57,NaN
18,Lorengau,PG,80.26,broken clouds,-2.02,147.27,Lorengau Harbourside Hotel
21,Kununurra,AU,78.80,broken clouds,-15.77,128.73,Hotel Kununurra
24,Rikitea,PF,75.11,broken clouds,-23.12,-134.97,Pension Maro'i
32,Saint Marys,US,82.00,broken clouds,30.73,-81.55,Spencer House Inn
36,Samarai,PG,80.83,broken clouds,-10.62,150.67,Nuli Sapi
41,Zaraza,VE,85.59,light rain,9.35,-65.32,Hotel Zaraza


In [83]:
hotel_df.count()

City                   218
Country                218
Max Temp               218
Current Description    218
Lat                    218
Lng                    218
Hotel Name             204
dtype: int64

In [84]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   204
Country                204
Max Temp               204
Current Description    204
Lat                    204
Lng                    204
Hotel Name             204
dtype: int64

In [85]:
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Saint-Philippe,RE,75.20,broken clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
12,Bengkulu,ID,75.96,light rain,-3.80,102.27,Grage Hotel Bengkulu
13,Atuona,PF,77.99,light rain,-9.80,-139.03,Villa Enata
18,Lorengau,PG,80.26,broken clouds,-2.02,147.27,Lorengau Harbourside Hotel
21,Kununurra,AU,78.80,broken clouds,-15.77,128.73,Hotel Kununurra
24,Rikitea,PF,75.11,broken clouds,-23.12,-134.97,Pension Maro'i
32,Saint Marys,US,82.00,broken clouds,30.73,-81.55,Spencer House Inn
36,Samarai,PG,80.83,broken clouds,-10.62,150.67,Nuli Sapi
41,Zaraza,VE,85.59,light rain,9.35,-65.32,Hotel Zaraza
47,Tocok,PH,78.80,broken clouds,15.75,120.30,Dandan’s Garden & Farm Resort


In [86]:
# 8a. Create the output File (CSV)
output_data_file = "C:\\Users\\asus\\World_Weather_Analysis\\Vacation_Search\\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [87]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and<dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [89]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))